In [1]:
import torch, sys
sys.path.insert(0, '../')
from my_utils import gpu_utils
import importlib, gc
from my_utils.alignment_features import *
import my_utils.alignment_features as afeatures
importlib.reload(afeatures)
import gnn_utils.graph_utils as gutils

/mounts/Users/student/ayyoob/anaconda3/envs/multalign_graph/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# !pip install torch-geometric
# !pip install tensorboardX

# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip

#  print(torch.version.cuda)
#  print(torch.__version__)    

dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [3]:

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt




In [4]:
from my_utils import align_utils as autils, utils
import argparse
from multiprocessing import Pool
import random

# set random seed
config_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc-ui-demo/config_pbc.ini"
utils.setup(config_file)

params = argparse.Namespace()


params.gold_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi-grc-fin-gold-alignments_train.txt"
pros, surs = autils.load_gold(params.gold_file)
all_verses = list(pros.keys())
params.gold_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi-heb-fin-gold-alignments_train.txt"
pros, surs = autils.load_gold(params.gold_file)
all_verses.extend(list(pros.keys()))
all_verses = list(set(all_verses))
print(len(all_verses))

params.editions_file =  "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi_lang_list.txt"
editions, langs = autils.load_simalign_editions(params.editions_file)
current_editions = [editions[lang] for lang in langs]

def get_pruned_verse_alignments(args):
    verse, current_editions = args
    
    verse_aligns_inter = autils.get_verse_alignments(verse)
    verse_aligns_gdfa = autils.get_verse_alignments(verse, gdfa=True)

    autils.prune_non_necessary_alignments(verse_aligns_inter, current_editions)
    autils.prune_non_necessary_alignments(verse_aligns_gdfa, current_editions)

    gc.collect()
    return verse_aligns_inter, verse_aligns_gdfa
    

verse_alignments_inter = {}
verse_alignments_gdfa = {}
args = []
for i,verse in enumerate(all_verses):
    args.append((verse, current_editions[:]))

#print('going to get alignments')
#with Pool(20) as p:
#    all_res = p.map(get_pruned_verse_alignments, args)

#for i,verse in enumerate(all_verses):
#    verse_aligns_inter, verse_aligns_gdfa = all_res[i]
    
    #verse_alignments_inter[verse] = verse_aligns_inter
    #verse_alignments_gdfa[verse] = verse_aligns_gdfa

#utils.LOG.info("done reading alignments")
#torch.save(verse_alignments_inter, "/mounts/work/ayyoob/models/gnn/pruned_alignments_train_inter.pickle")
#torch.save(verse_alignments_gdfa, "/mounts/work/ayyoob/models/gnn/pruned_alignments_train_gdfa.pickle")
#utils.LOG.info('done saving pruned alignments')

print('reading inter verse alignments')
#verse_alignments_inter = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_train_inter_8000.pickle")
gc.collect()
print('done reading inter verse alignments')

#for verse in all_verses[:]:
#    if len(verse_alignments_inter[verse].keys()) < 10:
#        all_verses.remove(verse)

24159
reading inter verse alignments
done reading inter verse alignments


In [5]:
class Discriminator(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Discriminator, self).__init__()
        self.lin1 = torch.nn.Linear(in_channels, hidden_channels)
        self.lin2 = torch.nn.Linear(hidden_channels, hidden_channels)
        self.lin3 = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        return x


In [6]:
class Encoder2(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Encoder2, self).__init__()
        self.conv1 = pyg_nn.GATConv(in_channels, 2*out_channels)
        self.conv2 = pyg_nn.GATConv(2 * out_channels , out_channels)

    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index, ))
        return self.conv2(x, edge_index)

class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels, features, n_head = 2, edge_feature_dim = 0,):
        super(Encoder, self).__init__()
        #self.lin = nn.Linear(in_channels, out_channels)
        self.conv1 = pyg_nn.GATConv(in_channels, 2*out_channels, heads= n_head)
        self.conv2 = pyg_nn.GATConv(2 * n_head *  out_channels , out_channels, heads= 1)
        #self.conv3 = pyg_nn.GATConv(2 * n_head *  out_channels , out_channels, heads= n_head)
        self.f_embedding = nn.Linear(in_channels, in_channels)
        

        self.feature_encoder = afeatures.FeatureEncoding(features, word_vectors)
        #self.already_inited = False
        #self.prev_edge_index = None
        #self.prev_edge_attr = None

    def forward(self, x, edge_index):
        x = self.feature_encoder(x, dev)
        x = F.relu(self.f_embedding(x))
        #if not self.already_inited or self.prev_edge_index.data_ptr() != edge_index.data_ptr():
        #    edge_index_np = edge_index.cpu().numpy()
        #    val_indices = x_edge_np[edge_index_np[0, :], edge_index_np[1, :]]
        #    vals = x_edge_vals[val_indices, :]
        #    vals = vals.reshape((vals.shape[1], vals.shape[2]))
        #    self.prev_edge_attr = vals.to(dev)
        #    self.prev_edge_index = edge_index
        #    self.already_inited = True
        #x = self.lin(x)
        x = F.elu(self.conv1(x, edge_index, ))
        #x = self.conv_gin(x, edge_index)
        #x = F.elu(self.conv2(x, edge_index))
        return self.conv2(x, edge_index)#, self.conv3(x, edge_index)

In [7]:
def clean_memory():
    gc.collect()
    with torch.no_grad():
        torch.cuda.empty_cache()

def train(epoch):
    global optimizer
    total_loss = 0
    cluster_loss = 0
    model.train()

    #for i in tqdm(range(int(train_pos_edge_index_permed.shape[1]/batch_size)+1)):
    for i,batch_ in enumerate(tqdm(data_loader)):
        for verse in batch_:
            if verse in masked_verses:
                continue
            batch = batch_[verse]
            optimizer.zero_grad()
            
            x = batch['x'].to(dev)
            edge_index = batch['edge_index'].to(dev)
            if torch.max(edge_index) >= x.shape[0]:
                print(torch.max(edge_index), x.shape)
                print(batch)
                break

            try:
                z = model.encode(x, edge_index)
            except Exception as e:
                global sag, khar, gav
                sag, khar, gav =  (i, batch_, verse)
                print(e)
                1/0
            #z1 = encoder2(z, torch.tensor(batch['intra_sent_edges'], dtype=torch.long).to(dev))
            #z = torch.cat((z,z1), dim=1)
            #for j in range(5):
            #    discriminator_optimizer.zero_grad()
            #    discriminator_loss = model.discriminator_loss(z) / (int(train_pos_edge_index_permed.shape[1]/batch_size)+1)
            #    discriminator_loss.backward()
            #    discriminator_optimizer.step()
            
            pos = torch.tensor(batch['pos'], dtype=torch.long).to(dev)
            neg = torch.tensor(batch['neg'], dtype=torch.long).to(dev)
            #nodes = torch.tensor(list(batch['nodes']), dtype=torch.long).to(dev)

            loss1 = model.recon_loss( z, pos, neg) #TODO try providing better neg edges
            #ortho_loss, mincut_loss, entropy_loss = model.decoder.clustering_loss(z, nodes, batch['adjacency'])
            
            loss =   loss1 * pos.shape[1] #+ ortho_loss + mincut_loss #+ 0.05 * entropy_loss #* pos.shape[1]/train_neg_edge_index.shape[1] #+ model.reg_loss(z)/(int(train_pos_edge_index_permed.shape[1]/batch_size)+1)# + (1 / x.shape[0]) * model.kl_loss()

            loss.backward()
            optimizer.step()
            total_loss += loss.item() 
            cluster_loss += loss1

            
            #if i  % 3000 == 2999:
            #    #alignment_test(epoch, test_dataset.edge_index, editf1, editf2, test_verses, test_nodes_map,
            #    #        dev, model, x_test, pros, surs, verse_alignments_inter, verse_alignments_gdfa, writer, gnn_dataset.verse_info)

            #    clean_memory()
                
            #    eval_utils.alignment_test(epoch, grc_test_dataset.edge_index, editf_fin, editf_grc, grc_test_verses[:], grc_test_dataset.nodes_map,
            #            dev, model, grc_test_dataset.x, pros_grc, surs_grc, grc_test_verse_alignments_inter, grc_test_verse_alignments_gdfa, writer, gnn_dataset_grc.verse_info)

            #    eval_utils.alignment_test(epoch, heb_test_dataset.edge_index, editf_fin, editf_heb, heb_test_verses[:], heb_test_dataset.nodes_map,
            #                            dev, model, heb_test_dataset.x, pros_heb, surs_heb, heb_test_verse_alignments_inter, heb_test_verse_alignments_gdfa, writer, gnn_dataset_heb.verse_info)

            #    eval_utils.alignment_test(epoch, blinker_test_dataset.edge_index, editf12, editf22, blinker_verses, blinker_test_dataset.nodes_map,
            #                dev, model, blinker_test_dataset.x, pros_blinker, surs_blinker, blinker_verse_alignments_inter, blinker_verse_alignments_gdfa, writer, gnn_dataset_blinker.verse_info)
                
            #    clean_memory()   
            #    # decoder.set_objective('sequence_prediction')
            #    # auc, ap = test(edge_index_seq_sent, edge_index_seq_sent_neg, epoch)
            #    # print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
            #    # if epoch > 4:
            #    #     decoder.set_objective('link_prediction')
            #    model.train()
            
            #if (i+1)*batch_size > train_pos_edge_index.shape[1]:
            #    break

            #if i % 51 == 0:
            #    clean_memory()
    
    writer.add_scalar("loss", total_loss, epoch)
    print(f"train loss: {total_loss}")
    print(f"cluster loss: {cluster_loss}")

def test(pos_edge_index, neg_edge_index, epoch):
    model.eval()
    tot_auc = tot_ap = 0
    with torch.no_grad():
        z = model.encode(x_test, torch.cat((train_pos_edge_index, neg_edge_index), dim=1).to(dev))
        
        neg_pos_coeff = neg_edge_index.shape[1]/ pos_edge_index.shape[1]
        for i in (range(int(pos_edge_index.shape[1]/batch_size)+1)):
            auc,ap = model.test(z, pos_edge_index[:, i*batch_size:(i+1)*batch_size].to(dev),
                                neg_edge_index[:, int(i*batch_size*neg_pos_coeff):int((i+1)*batch_size*neg_pos_coeff)].to(dev))

            tot_auc += auc * pos_edge_index[:, i*batch_size:(i+1)*batch_size].shape[1]
            tot_ap += ap *  pos_edge_index[:, i*batch_size:(i+1)*batch_size].shape[1]


    return tot_auc/pos_edge_index.shape[1], tot_ap/pos_edge_index.shape[1]



In [8]:
EPS = 1e-15

def _diag(x):
    eye = torch.eye(x.size(0)).type_as(x)
    out = eye * x.unsqueeze(1).expand(x.size(0), x.size(0))
    return out

class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, edge_features, n_cluster=32):
        super(Decoder, self).__init__()
        #self.feature_encoder = afeatures.FeatureEncoding(edge_features)
        self.features_size = sum([x.out_dim for x in edge_features])
        self.representataion_size = (input_size - self.features_size)

        self.transfer = nn.Sequential(nn.Linear(input_size, hidden_size*2), nn.ReLU(), nn.Dropout(drop_out),
                            nn.Linear(hidden_size*2, hidden_size), nn.ReLU(), nn.Dropout(drop_out),
                            nn.Linear(hidden_size, 1))

        #self.transfer = nn.Sequential(nn.ELU(), nn.Linear(n_cluster*2, 1), nn.ELU())

        #self.n_cluster = n_cluster                
        #self.cluster = nn.Sequential(nn.Linear(int((input_size - len(edge_features))/2), hidden_size*2), nn.ELU(), nn.Linear(hidden_size*2, 2*n_cluster))
        #self.actual_cluster = nn.Linear(2*n_cluster, n_cluster)
        #self.cos = nn.CosineSimilarity(dim=1)        
        #self.dist = nn.PairwiseDistance()
        #self.gnn_transform = nn.Sequential(nn.Linear(self.representataion_size, hidden_size), nn.ReLU(), nn.Dropout(drop_out))
        self.counter = 0

        self.objective = 'link_prediction'
    def forward(self, z, edge_index, sigmoid = True):
        if self.features_size > 0:
            if self.objective == 'link_prediction':
                edge_index_np = edge_index.cpu().numpy()
                val_indices = x_edge_np[edge_index_np[0, :], edge_index_np[1, :]]
                val_indices = np.squeeze(np.asarray(val_indices))
                vals = x_edge_vals2[val_indices, :]
            elif self.objective == 'sequence_prediction':
                vals = torch.zeros((edge_index.shape[1], self.features_size)).to(dev)


            features = self.feature_encoder(vals.to(dev), dev)
            #features = vals.to(dev)
            h1 = z[edge_index[0, :]]
            h2 = z[edge_index[1, :]]

            self.counter += 1

            #rep = self.gnn_transform(torch.cat((h1, h2), dim=1))
            res = self.transfer(torch.cat((self.cluster(h1), self.cluster(h2), features), dim=1))
            #res = self.transfer(features)
        else:
            h1 = z[edge_index[0, :]]
            h2 = z[edge_index[1, :]]

            
            res = self.transfer(torch.cat((h1, h2), dim=-1))

            #res = self.transfer(torch.cat((self.cluster(h1), self.cluster(h2)), dim=1))
            #res = torch.sum(torch.pow(F.softmax(self.cluster(h1)/1, dim=1) - F.softmax(self.cluster(h2)/1, dim=1), 2), dim=1)
            #res = self.cos(self.cluster(h1), self.cluster(h2))
            #res = - self.dist(self.cluster(h1), self.cluster(h2))
            #print(res)
        res = torch.sigmoid(res) if sigmoid else res
        return res

    def set_objective(self, objective):
        self.objective = objective
        
    def clustering_loss(self, z, nodes, adjacency):
        s = self.actual_cluster(torch.relu(self.cluster(z[nodes])))
        s = torch.softmax(s, dim=-1)
        entropy_loss = (-s * torch.log(s + EPS)).sum(dim=-1).mean()

        ss = torch.matmul(s.transpose(0, 1), s)
        i_s = torch.eye(self.n_cluster).type_as(ss)
        ortho_loss = torch.norm(
            ss / torch.norm(ss, dim=(-1, -2), keepdim=True) -
            i_s / torch.norm(i_s), dim=(-1, -2))
        ortho_loss = torch.mean(ortho_loss)

        adjacency = adjacency.to(dev).float()
        out_adj = torch.matmul(s.transpose(0, 1),torch.sparse.mm(adjacency, s))
        # MinCUT regularization.
        mincut_num = torch.trace(out_adj)
        #d_flat = torch.einsum('ij->i', adjacency) # FIXME since I don't consider the whole adjacency matrix this could be a source of problem
        d_flat = torch.sparse.sum(adjacency, dim=1).to_dense()
        d = _diag(d_flat)
        mincut_den = torch.trace(
            torch.matmul(torch.matmul(s.transpose(0, 1), d), s))
        mincut_loss = -(mincut_num / mincut_den)
        mincut_loss = torch.mean(mincut_loss)

        return ortho_loss, mincut_loss, entropy_loss
    
    def get_alignments(self, z, edge_index):
        h1 = z[edge_index[0, :]]
        h2 = z[edge_index[1, :]]
        
        h1 = torch.softmax(self.cluster(h1), dim=1)
        h2 = torch.softmax(self.cluster(h2), dim=1)

        h1_max = torch.argmax(h1, dim=1)
        h2_max = torch.argmax(h2, dim=1)

        h1_cluster = torch.zeros(*h1.shape)
        h2_cluster = torch.zeros(*h2.shape)

        h1_cluster[range(h1.size(0)), h1_max] = 1
        h2_cluster[range(h2.size(0)), h2_max] = 1
        res = torch.max(h1_cluster * h2_cluster, dim=1).values

        #res = h1 * h2
        #res = torch.sum(res, dim = 1)
        return torch.unsqueeze(res, dim=1)

ZeroDivisionError: division by zero

In [ ]:
import pickle

train_verses = all_verses[:]
test_verses = all_verses[:] 
editf1 = 'fin-x-bible-helfi'
editf2 = "heb-x-bible-helfi"


if 'jpn-x-bible-newworld' in  current_editions[:]:
     current_editions.remove('jpn-x-bible-newworld')
if 'grc-x-bible-unaccented' in  current_editions[:]:
     current_editions.remove('grc-x-bible-unaccented')
def get_train_dataset():
     return torch.load("/mounts/work/ayyoob/models/gnn/dataset_helfi_train_community_word_8000.pickle", map_location=torch.device('cpu'))
#train_dataset = get_train_dataset()
#train_dataset, train_nodes_map = create_dataset(train_verses, verse_alignments_inter, small_editions)
features = train_dataset.features
train_nodes_map = train_dataset.nodes_map
#edge_index_intra_sent = train_dataset.edge_index_intra_sent
#test_edge_index_intra_sent = edge_index_intra_sent

# test_dataset, test_nodes_map = create_dataset(test_verses, verse_alignments_inter, small_editions)
test_dataset, test_nodes_map = train_dataset, train_nodes_map
test_verses = train_verses
print(train_dataset.x.shape)

# gutils.augment_features(test_dataset)
# x_edge, features_edge = gutils.create_edge_attribs(train_nodes_map, train_verses, small_editions, verse_alignments_inter, train_dataset.x.shape[0])
# with open("./dataset.pickle", 'wb') as of:
#     pickle.dump(train_dataset, of)
gc.collect()


torch.Size([10331043, 10])


0

In [ ]:
from gensim.models import Word2Vec
w2v_model = Word2Vec.load("/mounts/work/ayyoob/models/w2v/word2vec_helfi_langs_15e.model")

print(w2v_model.wv.vectors.shape)

word_vectors = torch.from_numpy(w2v_model.wv.vectors).float()

print(word_vectors.shape)

(2354770, 100)
torch.Size([2354770, 100])


In [ ]:
# edges_intra_sent, edges_seq_sent = get_inter_sentence_connections(train_dataset.nodes_map)
# edge_index_seq_sent = torch.tensor(edges_seq_sent, dtype=torch.long)
# train_dataset.edge_index_seq_sent = edge_index_seq_sent
# torch.cuda.set_device(int(free_gpu1))
# edge_index_intra_sent = torch.tensor(edges_intra_sent, dtype=torch.long).to(dev)
# train_dataset.edge_index_intra_sent = edge_index_intra_sent
# test_edge_index_intra_sent = train_dataset.edge_index_intra_sent
# print(train_dataset.edge_index_intra_sent.shape)

In [ ]:
import pickle
import torch
## with open("./features_edge.pickle", 'wb') as of:
##     pickle.dump(features_edge, of)

## print('done first')
## with open("/mounts/work/ayyoob/models/gnn//x_edge.pickle", 'wb') as of:
##     pickle.dump(x_edge, of)

#with open("./features_edge.pickle", 'rb') as inf:
#    features_edge = pickle.load(inf)


## indices = [[],[]]
## values = []

## print('going to create sparse matrix representation')
## for i in range(len(ss)):
##     print(i)
##     for j in range(len(ss)):
##         if ss[i][j] != None and ss[i][j] != []:
##             indices[0].append(i)
##             indices[1].append(j)
##             values.append(ss[i][j])


## with open("./edge_attribs_sparse_indices.pickle", 'wb') as of:
##     pickle.dump(indices, of)

## with open("./edge_attribs_sparse_values.pickle", 'wb') as of:
##     pickle.dump(values, of)

## print('loading indices')
## with open("./edge_attribs_sparse_indices.pickle", 'rb') as inf:
##     indices = pickle.load(inf)

## print('loading values')
## with open("./edge_attribs_sparse_values.pickle", 'rb') as inf:
##     values = pickle.load(inf)

## print('creating sparse tensor')
## s = torch.sparse_coo_tensor(indices, values, (67800, 67800, len(ff)), dtype=torch.float16)
## print('saving sparse matrix')
## torch.save(s, "/mounts/work/ayyoob/models/gnn/edge_attribs_tensor16.pickle")

#print('loading sparse matrix')
#x_edge = torch.load("/mounts/work/ayyoob/models/gnn/edge_attribs_tensor.pickle")


#train_dataset.features_edge = features_edge



In [ ]:
from scipy.sparse import csr_matrix
import numpy as np

#x_edge = x_edge.coalesce()
#torch.cuda.set_device(1)
#x_edge_vals = x_edge.values()
#indices_np = x_edge.indices().numpy()
#print(indices_np.shape)
#x_edge_np = csr_matrix((np.arange(indices_np.shape[1]), (indices_np[0, :], indices_np[1,:])), shape=(67800, 67800))

##x_edge_vals = x_edge_vals.cpu()
##maxes = torch.max(x_edge_vals,0)
##mins = torch.min(x_edge_vals,0)
##x_edge_vals_d = torch.div(x_edge_vals, maxes.values)

#print('creating targets')
#targets = torch.zeros(indices_np.shape[1], dtype=torch.int64)
#pos_indices = x_edge_np[train_dataset.edge_index.cpu().numpy()[0,:], train_dataset.edge_index.cpu().numpy()[1,:]]
#pos_indices = np.squeeze(np.asarray(pos_indices))
#targets[pos_indices] = 1
#print("done")

In [ ]:
 # run on delta, extract w2v features
#sys.path.insert(0, '../')
#import pickle
#from gensim.models import Word2Vec
#from app.document_retrieval import DocumentRetriever
#from my_utils import utils
#config_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc-ui-demo/config_pbc.ini"
#utils.setup(config_file)
#import torch
#import my_utils.alignment_features as feat_utils

#doc_retriever = DocumentRetriever()

#model_w2v = Word2Vec.load("word2vec_83langs_15epoch.model")
#train_dataset = torch.load("/mounts/work/ayyoob/models/gnn/dataset_en_fr_full.pickle")
#nodes_map = train_dataset.nodes_map

#x = [[] for i in range(train_dataset.x.shape[0])]
#for edition_f in nodes_map:
#    utils.LOG.info(f"processing edition {edition_f}")
#    for verse in nodes_map[edition_f]:         #toknom nodecount
#        line = doc_retriever.retrieve_document(f'{verse}@{edition_f}')
#        line = line.strip().split()

#        for tok in nodes_map[edition_f][verse]:
#            w_emb = model_w2v.wv[f'{edition_f[:3]}:{line[tok]}']
#            x[nodes_map[edition_f][verse][tok]].extend(w_emb)

#x = torch.tensor(x, dtype=torch.float)
#train_dataset.x = torch.cat((train_dataset.x, x), dim=1)
#train_dataset.features.append(feat_utils.ForwardFeature(50, 100, 'W2v'))

#print(x.shape, train_dataset.x.shape, len(train_dataset.features))

#torch.save(train_dataset, "/mounts/work/ayyoob/models/gnn/dataset_en_fr_full.pickle")

In [ ]:
## Add node embedding features
#importlib.reload(gutils)
#x_,features_ = gutils.get_embedding_node_features(train_dataset.nodes_map, train_verses, small_editions, verse_alignments_inter, x_edge_np, x_edge_vals.cpu().numpy())
#train_dataset.x = torch.cat((train_dataset.x,x_), dim=1)
#train_dataset.features.extend(features_)

In [9]:
def get_blinker_dataset():
    return torch.load("/mounts/work/ayyoob/models/gnn/dataset_blinker_full_community_word.pickle", map_location=torch.device('cpu'))
    
blinker_test_dataset = get_blinker_dataset()
editf12 = "eng-x-bible-mixed"
editf22 = 'fra-x-bible-louissegond'

test_gold_eng_fra = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/eng_fra_pbc/eng-fra.gold"

pros_blinker, surs_blinker = autils.load_gold(test_gold_eng_fra)
blinker_verses = list(pros_blinker.keys())

#blinker_verse_alignments_inter = {}
#blinker_verse_alignments_gdfa = {}
#args = []
#for i,verse in enumerate(blinker_verses):
#    args.append((verse, current_editions))

#with Pool(20) as p:
#    all_res = p.map(get_pruned_verse_alignments, args)

#for i,verse in enumerate(blinker_verses):
#    verse_aligns_inter, verse_aligns_gdfa = all_res[i]
    
#    blinker_verse_alignments_inter[verse] = verse_aligns_inter
#    blinker_verse_alignments_gdfa[verse] = verse_aligns_gdfa

utils.LOG.info("done reading alignments")
#torch.save(blinker_verse_alignments_inter, "/mounts/work/ayyoob/models/gnn/pruned_alignments_blinker_inter.pickle")
#torch.save(blinker_verse_alignments_gdfa, "/mounts/work/ayyoob/models/gnn/pruned_alignments_blinker_gdfa.pickle")
utils.LOG.info('done saving pruned alignments')

print('reading inter verse alignments')
blinker_verse_alignments_inter = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_blinker_inter.pickle")
blinker_verse_alignments_gdfa = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_blinker_gdfa.pickle")
gc.collect()
print('done reading inter verse alignments')

verses_map = {}

for edit in blinker_test_dataset.nodes_map:
    for verse in blinker_test_dataset.nodes_map[edit]:
        if verse not in verses_map:
            for tok in blinker_test_dataset.nodes_map[edit][verse]:
                verses_map[verse] = blinker_test_dataset.nodes_map[edit][verse][tok]
                break

sorted_verses = sorted(verses_map.items(), key = lambda x: x[1])
blinker_verses = [item[0] for item in sorted_verses]


2021-09-11 11:13:09,970 - analytics - INFO - done reading alignments
2021-09-11 11:13:09,972 - analytics - INFO - done saving pruned alignments


reading inter verse alignments
done reading inter verse alignments


In [ ]:
#helfi_heb_dataset = torch.load("/mounts/work/ayyoob/models/gnn/dataset_helfi_heb_test_community_word.pickle", map_location=torch.device('cpu'))
#editf_fin = "fin-x-bible-helfi"
#editf_heb = 'heb-x-bible-helfi'

#test_gold_helfi_heb = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi-heb-fin-gold-alignments_test.txt"

#pros_heb, surs_heb = autils.load_gold(test_gold_helfi_heb)
#heb_verses = list(pros_blinker.keys())

##blinker_verse_alignments_inter = {}
##blinker_verse_alignments_gdfa = {}
##args = []
##for i,verse in enumerate(blinker_verses):
##    args.append((verse, current_editions))

##with Pool(20) as p:
##    all_res = p.map(get_pruned_verse_alignments, args)

##for i,verse in enumerate(blinker_verses):
##    verse_aligns_inter, verse_aligns_gdfa = all_res[i]
    
##    blinker_verse_alignments_inter[verse] = verse_aligns_inter
##    blinker_verse_alignments_gdfa[verse] = verse_aligns_gdfa

#utils.LOG.info("done reading alignments")
##torch.save(blinker_verse_alignments_inter, "/mounts/work/ayyoob/models/gnn/pruned_alignments_blinker_inter.pickle")
##torch.save(blinker_verse_alignments_gdfa, "/mounts/work/ayyoob/models/gnn/pruned_alignments_blinker_gdfa.pickle")
#utils.LOG.info('done saving pruned alignments')

#print('reading inter verse alignments')
#blinker_verse_alignments_inter = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_blinker_inter.pickle")
#blinker_verse_alignments_gdfa = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_blinker_gdfa.pickle")
#gc.collect()
#print('done reading inter verse alignments')

#verses_map = {}

#for edit in blinker_test_dataset.nodes_map:
#    for verse in blinker_test_dataset.nodes_map[edit]:
#        if verse not in verses_map:
#            for tok in blinker_test_dataset.nodes_map[edit][verse]:
#                verses_map[verse] = blinker_test_dataset.nodes_map[edit][verse][tok]
#                break

#sorted_verses = sorted(verses_map.items(), key = lambda x: x[1])
#blinker_verses = [item[0] for item in sorted_verses]

In [ ]:
#importlib.reload(afeatures)
#grc_test_dataset = torch.load("/mounts/work/ayyoob/models/gnn/dataset_helfi_grc_test_community_word.pickle", map_location=torch.device('cpu'))
#editf_fin = "fin-x-bible-helfi"
#editf_grc = 'grc-x-bible-helfi'

#test_gold_grc = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi-grc-fin-gold-alignments_test.txt"

#pros_grc, surs_grc = autils.load_gold(test_gold_grc)
#grc_verses = list(pros_grc.keys())


#grc_test_verse_alignments_inter = {}
#grc_test_verse_alignments_gdfa = {}
#gc.collect()
##args = []
##for i,verse in enumerate(grc_verses):
##    args.append((verse, current_editions))

##with Pool(20) as p:
##    all_res = p.map(get_pruned_verse_alignments, args)

##for i,verse in enumerate(grc_verses):
##    verse_aligns_inter, verse_aligns_gdfa = all_res[i]
    
##    grc_test_verse_alignments_inter[verse] = verse_aligns_inter
##    grc_test_verse_alignments_gdfa[verse] = verse_aligns_gdfa

#utils.LOG.info("done reading alignments")
##torch.save(grc_test_verse_alignments_inter, "/mounts/work/ayyoob/models/gnn/pruned_alignments_grc_inter.pickle")
##torch.save(grc_test_verse_alignments_gdfa, "/mounts/work/ayyoob/models/gnn/pruned_alignments_grc_gdfa.pickle")
#utils.LOG.info('done saving pruned alignments')

#print('reading inter verse alignments')
#grc_test_verse_alignments_inter = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_grc_inter.pickle")
#grc_test_verse_alignments_gdfa = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_grc_gdfa.pickle")
#gc.collect()
#print('done reading inter verse alignments')

#verses_map = {}

#for edit in grc_test_dataset.nodes_map:
#    for verse in grc_test_dataset.nodes_map[edit]:
#        if verse not in verses_map:
#            for tok in grc_test_dataset.nodes_map[edit][verse]:
#                verses_map[verse] = grc_test_dataset.nodes_map[edit][verse][tok]
#                break

#sorted_verses = sorted(verses_map.items(), key = lambda x: x[1])
#grc_test_verses = [item[0] for item in sorted_verses]

#gc.collect()

In [ ]:
#heb_test_dataset = torch.load("/mounts/work/ayyoob/models/gnn/dataset_helfi_heb_test_community_word.pickle", map_location=torch.device('cpu'))
#editf_fin = "fin-x-bible-helfi"
#editf_heb = 'heb-x-bible-helfi'

#test_gold_heb = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi-heb-fin-gold-alignments_test.txt"

#pros_heb, surs_heb = autils.load_gold(test_gold_heb)
#heb_verses = list(pros_heb.keys())


#heb_test_verse_alignments_inter = {}
#heb_test_verse_alignments_gdfa = {}
##args = []
##for i,verse in enumerate(heb_verses):
##    args.append((verse, current_editions))

##with Pool(20) as p:
##    all_res = p.map(get_pruned_verse_alignments, args)

##for i,verse in enumerate(heb_verses):
##    verse_aligns_inter, verse_aligns_gdfa = all_res[i]
    
##    heb_test_verse_alignments_inter[verse] = verse_aligns_inter
##    heb_test_verse_alignments_gdfa[verse] = verse_aligns_gdfa

##utils.LOG.info("done reading alignments")
##torch.save(heb_test_verse_alignments_inter, "/mounts/work/ayyoob/models/gnn/pruned_alignments_heb_inter.pickle")
##torch.save(heb_test_verse_alignments_gdfa, "/mounts/work/ayyoob/models/gnn/pruned_alignments_heb_gdfa.pickle")
##utils.LOG.info('done saving pruned alignments')

#print('reading inter verse alignments')
#heb_test_verse_alignments_inter = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_heb_inter.pickle")
#heb_test_verse_alignments_gdfa = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_heb_gdfa.pickle")
#gc.collect()
#print('done reading inter verse alignments')

#verses_map = {}

#for edit in heb_test_dataset.nodes_map:
#    for verse in heb_test_dataset.nodes_map[edit]:
#        if verse not in verses_map:
#            for tok in heb_test_dataset.nodes_map[edit][verse]:
#                verses_map[verse] = heb_test_dataset.nodes_map[edit][verse][tok]
#                break

#sorted_verses = sorted(verses_map.items(), key = lambda x: x[1])
#heb_test_verses = [item[0] for item in sorted_verses]
#gc.collect()

In [ ]:
verses_map = {}

for edit in train_dataset.nodes_map:
    for verse in train_dataset.nodes_map[edit]:
        if verse not in verses_map:
            for tok in train_dataset.nodes_map[edit][verse]:
                verses_map[verse] = train_dataset.nodes_map[edit][verse][tok]
                break

sorted_verses = sorted(verses_map.items(), key = lambda x: x[1])
all_verses = [item[0] for item in sorted_verses]

long_verses = set()

for edit in train_dataset.nodes_map.keys():
    for verse in train_dataset.nodes_map[edit]:
        to_print = False
        for tok in train_dataset.nodes_map[edit][verse]:
            if tok > 150:
                to_print = True
        if to_print == True:
            long_verses.add(verse)


train_verses = all_verses[:]

masked_verses = list(long_verses)
masked_verses.extend(blinker_verses)


In [ ]:
#verse_alignments_inter_1000 = {}
#for verse in train_verses[:8000]:
#    verse_alignments_inter_1000[verse] = verse_alignments_inter[verse]

#max_node = 0
#remaining_verses = train_verses[8000:]
#for editf in train_dataset.nodes_map:
#    for verse in remaining_verses:
#        if verse in train_dataset.nodes_map[editf]:
#            del train_dataset.nodes_map[editf][verse]

#    if train_verses[7999] in train_dataset.nodes_map[editf]:
#        max_ = max(list(train_dataset.nodes_map[editf][train_verses[7999]].values()))
#        max_node = max(max_, max_node)

#train_dataset.edge_index = train_dataset.edge_index[:, :int(879904916/2)]
#train_dataset.x = train_dataset.x[:max_node+2, :]

#torch.save(train_dataset, "/mounts/work/ayyoob/models/gnn/dataset_helfi_train_community_word_8000.pickle")
#torch.save(verse_alignments_inter_1000, "/mounts/work/ayyoob/models/gnn/pruned_alignments_train_inter_8000.pickle")

In [12]:
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import random

class GNNDataset(Dataset):

    def __init__(self, dataset, verses, edit_files, alignments, group_size = 360):
        self.verses = list(verses)
        self.edit_files = list(edit_files)
        self.nodes_map = dataset.nodes_map

        self.items = self.calculate_size(self.nodes_map, self.verses, self.edit_files, group_size)
        self.alignments = alignments
        self.verse_info = {}
        self.calculate_verse_stats(verses, edit_files, alignments, dataset)
    
    def calculate_size(self, nodes_map, verses, edit_files, group_size):
        res = []
        item = []
        self.not_presented = []
        for verse in verses:
            if len(item) > 0:
                res.append(item)
                item = []
            for i,editf1 in enumerate(edit_files):
                if editf1 not in nodes_map:
                    self.not_presented.append(editf1)
                    continue
                if verse in nodes_map[editf1]:
                    for editf2 in edit_files[i+1:]:
                        if editf2 not in nodes_map:
                            self.not_presented.append(editf2)
                            continue
                        if verse in nodes_map[editf2]:
                            item.append((verse, editf1, editf2))
                            if len(item) >= group_size:
                                res.append(item)
                                item = []
        
        if len(item)>0:
            res.append(item)
        
        print(f"not presented: {set(self.not_presented)}")

        return res

    def calculate_verse_stats(self,verses, edition_files, alignments, dataset):
        min_edge = 0
        for verse in tqdm(verses):
            min_nodes = 99999999999999
            max_nodes = 0
            #utils.LOG.info(f"adding {verse}")
            edges_tmp = [[],[]]
            x_tmp = []
            features = []
            for i,editf1 in enumerate(edition_files):
                for j,editf2 in enumerate(edition_files[i+1:]):
                    aligns = autils.get_aligns(editf1, editf2, alignments[verse])
                    if aligns != None:
                        for align in aligns:
                            try:
                                n1,_ = gutils.node_nom(verse, editf1, align[0], None, dataset.nodes_map, x_tmp, edition_files, features)
                                n2,_ = gutils.node_nom(verse, editf2, align[1], None, dataset.nodes_map, x_tmp, edition_files, features)
                                edges_tmp[0].extend([n1, n2])

                                max_nodes = max(n1, n2, max_nodes)
                                min_nodes = min(n1, n2, min_nodes)
                            except Exception as e:
                                print(editf1, editf2, verse)
                                raise(e)

            self.verse_info[verse] = {}

            self.verse_info[verse]['padding'] = min_nodes

            self.verse_info[verse]['x'] = dataset.x[min_nodes:max_nodes+1,:]
            
            self.verse_info[verse]['edge_index'] = dataset.edge_index[:, min_edge : min_edge + len(edges_tmp[0])] - min_nodes

            if torch.min(self.verse_info[verse]['edge_index']) != 0:
                print(verse, min_nodes, max_nodes, min_edge, len(edges_tmp[0]))
                print(torch.min(self.verse_info[verse]['edge_index']))
            
            if self.verse_info[verse]['x'].shape[0] != torch.max(self.verse_info[verse]['edge_index']) + 1 :
                print(verse, min_nodes, max_nodes, min_edge, len(edges_tmp[0]))
                print(torch.min(self.verse_info[verse]['edge_index']))
            
            min_edge = min_edge + len(edges_tmp[0])

            
    
    def __len__(self):
        #return self.length
        return len(self.items)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        item = self.items[idx]

        res_pos = [[],[]]
        res_neg = [[],[]]
        nodes = set()
        for instance in item:
            verse, editf1, editf2 = instance
            aligns = autils.get_aligns(editf1, editf2, self.alignments[verse]) 
            if aligns != None:
                for align in aligns:
                    p1, p2 = align
                    n1 = self.nodes_map[editf1][verse][p1] - self.verse_info[verse]['padding']
                    n2 = self.nodes_map[editf2][verse][p2] - self.verse_info[verse]['padding']
                    res_pos[0].extend([n1,n2])
                    res_pos[1].extend([n2,n1])

                    n2_ = random.choice( list(self.nodes_map[editf2][verse].values()) ) - self.verse_info[verse]['padding']
                    n1_ = random.choice( list(self.nodes_map[editf1][verse].values()) ) - self.verse_info[verse]['padding']
                    
                    if n2_ != n2:
                        res_neg[0].extend([n1, n2_])
                        res_neg[1].extend([n2_, n1])
                    
                    if n1_ != n1:
                        res_neg[0].extend([n1_, n2])
                        res_neg[1].extend([n2, n1_])
                    
                    nodes.update([n1, n2, n1_, n2_])
                
        
        return {'pos':res_pos, 'neg':res_neg, 'nodes':nodes, 'verse':verse, 'editf1':editf1, 'editf2':editf2}

def collate_fun(input):
    res = {}
    #all_edits = {}
    for item in input:
        verse = item['verse'] 
        if verse not in res:
            res[verse] = {'pos': [[],[]], 'neg' : [[],[]],
                 'x':gnn_dataset.verse_info[verse]['x'], 'edge_index':gnn_dataset.verse_info[verse]['edge_index']
                 ,'intra_sent_edges':[[],[]]}
        
        res[verse]['pos'][0].extend(item['pos'][0])
        res[verse]['pos'][1].extend(item['pos'][1])

        res[verse]['neg'][0].extend(item['neg'][0])
        res[verse]['neg'][1].extend(item['neg'][1])

        #if verse not in all_edits:
        #    all_edits[verse] = []

        #if item['editf1'] not in all_edits[verse]:
        #    e = eval_utils.get_all_edges(verse, item['editf1'], train_dataset.nodes_map, gnn_dataset.verse_info)
        #    res[verse]['intra_sent_edges'][0].extend(e[0])
        #    res[verse]['intra_sent_edges'][1].extend(e[1])

        #if item['editf2'] not in all_edits[verse]:
        #    e = eval_utils.get_all_edges(verse, item['editf2'], train_dataset.nodes_map, gnn_dataset.verse_info)
        #    res[verse]['intra_sent_edges'][0].extend(e[0])
        #    res[verse]['intra_sent_edges'][1].extend(e[1])

    #nodes = list(nodes)
    #mapping = {node:pos for pos, node in enumerate(nodes)}
    ##indices = [[i for i in range(len(res_pos[0]))],[i for i in range(len(res_pos[1]))]]
    #indices = [[],[]]
    ##adjacency = torch.zeros((len(nodes), len(nodes)), dtype=torch.float)
    #for i in range(len(res_pos[0])):
    ##    adjacency[mapping[res_pos[0][i]], mapping[res_pos[1][i]]] = 1
    #    indices[0].append(mapping[res_pos[0][i]])
    #    indices[1].append(mapping[res_pos[1][i]])

    #adjacency = torch.sparse_coo_tensor(indices, [1 for i in range(len(res_pos[0]))], (len(nodes), len(nodes)))
    return res


#gnn_dataset_train = GNNDataset(train_dataset, train_verses, current_editions, verse_alignments_inter)
gnn_dataset_blinker = GNNDataset(blinker_test_dataset, blinker_verses, current_editions, blinker_verse_alignments_inter)
#gnn_dataset_heb = GNNDataset(heb_test_dataset, heb_test_verses, current_editions, heb_test_verse_alignments_inter)
#gnn_dataset_grc = GNNDataset(grc_test_dataset, grc_test_verses, current_editions, grc_test_verse_alignments_inter)

#len(gnn_dataset_train)
gc.collect()

  1%|          | 3/250 [00:00<00:10, 23.49it/s]

not presented: set()


100%|██████████| 250/250 [00:14<00:00, 17.53it/s]


34

In [15]:
from gnn_utils import eval_utils
train_dataset.train_mask = train_dataset.val_mask = train_dataset.test_mask = train_dataset.y = None
test_dataset.train_mask = test_dataset.val_mask = test_dataset.test_mask = test_dataset.y = None

torch.cuda.set_device(0)

NameError: name 'train_dataset' is not defined

In [13]:
def save_model(model, name):
    model.encoder.feature_encoder.feature_types[0] = afeatures.OneHotFeature(20, 83, 'editf')
    model.encoder.feature_encoder.feature_types[1] = afeatures.OneHotFeature(32, 150, 'position')
    model.encoder.feature_encoder.feature_types[2] = afeatures.FloatFeature(4, 'degree_centrality')
    model.encoder.feature_encoder.feature_types[3] = afeatures.FloatFeature(4, 'closeness_centrality')
    model.encoder.feature_encoder.feature_types[4] = afeatures.FloatFeature(4, 'betweenness_centrality')
    model.encoder.feature_encoder.feature_types[5] = afeatures.FloatFeature(4, 'load_centrality')
    model.encoder.feature_encoder.feature_types[6] = afeatures.FloatFeature(4, 'harmonic_centrality')
    model.encoder.feature_encoder.feature_types[7] = afeatures.OneHotFeature(32, 250, 'greedy_modularity_community')
    model.encoder.feature_encoder.feature_types[8] = afeatures.OneHotFeature(32, 250, 'community_2')
    model.encoder.feature_encoder.feature_types[9] = afeatures.MappingFeature(100, 'word')
    torch.save(model, f'/mounts/work/ayyoob/models/gnn/checkpoint/{name}' + datetime.now().strftime("%Y%m%d-%H%M%S-") + '.pickle')

In [18]:
verse_counts = {400:'trainsize_400_51220210911-015525-.pickle', 800:'trainsize_800_51220210911-032412-.pickle', 
1600:"trainsize_1600_51220210911-045657-.pickle", 3200:'trainsize_3200_51220210911-063319-.pickle', 6400:'trainsize_6400_51220210911-081557-.pickle'}

for verse_count in verse_counts:
    print(f'loading {verse_count}')
    model = torch.load('/mounts/work/ayyoob/models/gnn/checkpoint/' + verse_counts[verse_count])

    writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S-") + f"samett-chs-feat-")

    eval_utils.alignment_test(1, blinker_test_dataset.edge_index, editf12, editf22, blinker_verses, blinker_test_dataset.nodes_map,
                dev, model, blinker_test_dataset.x, pros_blinker, surs_blinker, blinker_verse_alignments_inter, blinker_verse_alignments_gdfa, writer, gnn_dataset_blinker.verse_info)
        
    clean_memory()

loading 400


100%|██████████| 250/250 [00:15<00:00, 15.84it/s]




intersection prec: 0.971, rec: 0.521, F1: 0.678, AER: 0.319
gdfa prec: 0.856, rec: 0.71, F1: 0.776, AER: 0.221
my_gdfa prec: 0.752, rec: 0.735, F1: 0.743, AER: 0.256
argmax prec: 0.932, rec: 0.664, F1: 0.775, AER: 0.221
resnorm prec: 0.41, rec: 0.787, F1: 0.539, AER: 0.474
itermax2-.9 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.95 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.8 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
my_gd prec: 0.893, rec: 0.706, F1: 0.789, AER: 0.208
my_gd_gdfa prec: 0.844, rec: 0.788, F1: 0.815, AER: 0.184
new1 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd_gdfa prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
loading 800


100%|██████████| 250/250 [00:10<00:00, 24.97it/s]




intersection prec: 0.971, rec: 0.521, F1: 0.678, AER: 0.319
gdfa prec: 0.856, rec: 0.71, F1: 0.776, AER: 0.221
my_gdfa prec: 0.777, rec: 0.751, F1: 0.764, AER: 0.236
argmax prec: 0.938, rec: 0.671, F1: 0.782, AER: 0.215
resnorm prec: 0.55, rec: 0.779, F1: 0.645, AER: 0.363
itermax2-.9 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.95 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.8 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
my_gd prec: 0.911, rec: 0.711, F1: 0.799, AER: 0.198
my_gd_gdfa prec: 0.856, rec: 0.789, F1: 0.821, AER: 0.177
new1 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd_gdfa prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
loading 1600


...



intersection prec: 0.971, rec: 0.521, F1: 0.678, AER: 0.319
gdfa prec: 0.856, rec: 0.71, F1: 0.776, AER: 0.221
my_gdfa prec: 0.778, rec: 0.752, F1: 0.765, AER: 0.235
argmax prec: 0.941, rec: 0.673, F1: 0.785, AER: 0.212
resnorm prec: 0.577, rec: 0.776, F1: 0.662, AER: 0.345
itermax2-.9 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.95 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.8 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
my_gd prec: 0.913, rec: 0.711, F1: 0.799, AER: 0.198
my_gd_gdfa prec: 0.856, rec: 0.787, F1: 0.82, AER: 0.178
new1 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd_gdfa prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
loading 3200


100%|██████████| 250/250 [00:10<00:00, 24.81it/s]




intersection prec: 0.971, rec: 0.521, F1: 0.678, AER: 0.319
gdfa prec: 0.856, rec: 0.71, F1: 0.776, AER: 0.221
my_gdfa prec: 0.781, rec: 0.753, F1: 0.767, AER: 0.232
argmax prec: 0.939, rec: 0.672, F1: 0.783, AER: 0.214
resnorm prec: 0.626, rec: 0.771, F1: 0.691, AER: 0.314
itermax2-.9 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.95 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.8 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
my_gd prec: 0.92, rec: 0.705, F1: 0.798, AER: 0.198
my_gd_gdfa prec: 0.862, rec: 0.787, F1: 0.823, AER: 0.176
new1 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd_gdfa prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
loading 6400


100%|██████████| 250/250 [00:09<00:00, 25.11it/s]




intersection prec: 0.971, rec: 0.521, F1: 0.678, AER: 0.319
gdfa prec: 0.856, rec: 0.71, F1: 0.776, AER: 0.221
my_gdfa prec: 0.786, rec: 0.757, F1: 0.771, AER: 0.228
argmax prec: 0.941, rec: 0.673, F1: 0.785, AER: 0.213
resnorm prec: 0.649, rec: 0.776, F1: 0.707, AER: 0.297
itermax2-.9 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.95 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.8 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
my_gd prec: 0.916, rec: 0.712, F1: 0.801, AER: 0.196
my_gd_gdfa prec: 0.86, rec: 0.789, F1: 0.823, AER: 0.175
new1 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd_gdfa prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0


In [ ]:
from tqdm import tqdm
features_edge = []
drop_out = 0
verse_counts = {25:120, 50:80, 100:64, 200:32, 400:16, 800:8, 1600:4, 3200:2, 6400:1}

gnn_dataset_blinker = GNNDataset(blinker_test_dataset, blinker_verses, current_editions, blinker_verse_alignments_inter)

for verse_count in verse_counts:
    print('training with less verses: ', verse_count)
    features = train_dataset.features
    gnn_dataset = GNNDataset(train_dataset, train_verses[:verse_count], current_editions, verse_alignments_inter)
    data_loader = DataLoader(gnn_dataset, batch_size=1, collate_fn=collate_fun, shuffle=True)

    clean_memory()

    n_head = 1

    channels = 512

    in_dim = sum(t.out_dim for t in features)
    decoder_in_dim = n_head * channels * 2 + sum(t.out_dim for t in features_edge) 
    print('edge features size: ', sum(t.out_dim for t in features_edge))

    decoder = Decoder(decoder_in_dim, int(decoder_in_dim/2), features_edge, n_cluster=64)
    model = pyg_nn.GAE(Encoder(in_dim, channels, features, n_head, edge_feature_dim=len(features_edge)), decoder).to(dev)

    print("sending input to gpu")
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
    optimizer.add_param_group({'params': word_vectors})

    writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S-") + f"samett-{channels}chs-feat{train_dataset.num_node_features}-")

    torch.set_printoptions(edgeitems=5)
    print("model params - decoder params - conv1", sum(p.numel() for p in model.parameters()), sum(p.numel() for p in decoder.parameters()))

    for epoch in range(1, verse_counts[verse_count]+1):
        print(f"\n----------------epoch {epoch} ---------------")
        train(epoch)
        
    save_model(model, f'trainsize_{verse_count}_512')
    clean_memory()
    eval_utils.alignment_test(epoch, blinker_test_dataset.edge_index, editf12, editf22, blinker_verses, blinker_test_dataset.nodes_map,
                dev, model, blinker_test_dataset.x, pros_blinker, surs_blinker, blinker_verse_alignments_inter, blinker_verse_alignments_gdfa, writer, gnn_dataset_blinker.verse_info)
        
    clean_memory()
    
    model = None
    decoder = None
    gnn_dataset_train = None
    gnn_dataset = None
    clean_memory()
    


  1%|          | 3/250 [00:00<00:08, 28.79it/s]

not presented: set()


 12%|█▏        | 3/25 [00:00<00:00, 23.20it/s]

training with less verses:  25
not presented: set()


100%|██████████| 25/25 [00:01<00:00, 16.10it/s]


edge features size:  0


  0%|          | 0/123 [00:00<?, ?it/s]

sending input to gpu
model params - decoder params - conv1 237901021 1574913

----------------epoch 1 ---------------


  1%|          | 1/123 [00:01<02:29,  1.23s/it]

In [ ]:
i = sag
batch = khar
verse = gav
print(i, verse)

keys = list(gnn_dataset.verse_info.keys())

gnn_dataset.verse_info[verse]


In [ ]:
data_loader_blinker = DataLoader(gnn_dataset_blinker, batch_size=1, collate_fn=collate_fun, shuffle=True)
data_loader_heb = DataLoader(gnn_dataset_heb, batch_size=1, collate_fn=collate_fun, shuffle=True)
data_loader_grc = DataLoader(gnn_dataset_grc, batch_size=1, collate_fn=collate_fun, shuffle=True)

clean_memory()
data_loader = data_loader_blinker
gnn_dataset = gnn_dataset_blinker
train(1)
clean_memory()
eval_utils.alignment_test(epoch, blinker_test_dataset.edge_index, editf12, editf22, blinker_verses[:], blinker_test_dataset.nodes_map,
                    dev, model, blinker_test_dataset.x, pros_blinker, surs_blinker, blinker_verse_alignments_inter, blinker_verse_alignments_gdfa, writer, gnn_dataset_blinker.verse_info)
clean_memory()

data_loader = data_loader_grc
gnn_dataset = gnn_dataset_grc
train(1)
clean_memory()
eval_utils.alignment_test(epoch, grc_test_dataset.edge_index, editf_fin, editf_grc, grc_test_verses[:], grc_test_dataset.nodes_map,
                        dev, model, grc_test_dataset.x, pros_grc, surs_grc, grc_test_verse_alignments_inter, grc_test_verse_alignments_gdfa, writer, gnn_dataset_grc.verse_info)
clean_memory()

data_loader = data_loader_heb
gnn_dataset = gnn_dataset_heb
train(1)
clean_memory()
eval_utils.alignment_test(epoch, heb_test_dataset.edge_index, editf_fin, editf_heb, heb_test_verses[:], heb_test_dataset.nodes_map,
                        dev, model, heb_test_dataset.x, pros_heb, surs_heb, heb_test_verse_alignments_inter, heb_test_verse_alignments_gdfa, writer, gnn_dataset_heb.verse_info)
clean_memory()


100%|██████████| 1180/1180 [00:32<00:00, 36.58it/s]


train loss: 0
cluster loss: 0


intersection prec: 0.971, rec: 0.521, F1: 0.678, AER: 0.319
gdfa prec: 0.856, rec: 0.71, F1: 0.776, AER: 0.221
my_gdfa prec: 0.783, rec: 0.754, F1: 0.768, AER: 0.231
argmax prec: 0.94, rec: 0.673, F1: 0.784, AER: 0.213
resnorm prec: 0.707, rec: 0.759, F1: 0.732, AER: 0.269
itermax2-.9 prec: 0.852, rec: 0.724, F1: 0.783, AER: 0.214
itermax2-.95 prec: 0.85, rec: 0.724, F1: 0.782, AER: 0.215
itermax2-.8 prec: 0.854, rec: 0.725, F1: 0.784, AER: 0.214
my_gd prec: 0.924, rec: 0.704, F1: 0.799, AER: 0.198
my_gd_gdfa prec: 0.865, rec: 0.784, F1: 0.823, AER: 0.175
new1 prec: 0.945, rec: 0.661, F1: 0.778, AER: 0.219
new_mygd prec: 0.92, rec: 0.706, F1: 0.799, AER: 0.198
new_mygd_gdfa prec: 0.863, rec: 0.786, F1: 0.823, AER: 0.176


 43%|████▎     | 2999/7000 [07:08<09:53,  6.74it/s]



intersection prec: 0.897, rec: 0.506, F1: 0.647, AER: 0.353
gdfa prec: 0.733, rec: 0.671, F1: 0.701, AER: 0.299
my_gdfa prec: 0.647, rec: 0.797, F1: 0.714, AER: 0.286
argmax prec: 0.857, rec: 0.697, F1: 0.769, AER: 0.231
resnorm prec: 0.679, rec: 0.779, F1: 0.726, AER: 0.275
itermax2-.9 prec: 0.736, rec: 0.766, F1: 0.751, AER: 0.25
itermax2-.95 prec: 0.734, rec: 0.764, F1: 0.749, AER: 0.251
itermax2-.8 prec: 0.738, rec: 0.766, F1: 0.752, AER: 0.248
my_gd prec: 0.838, rec: 0.731, F1: 0.781, AER: 0.22
my_gd_gdfa prec: 0.736, rec: 0.786, F1: 0.76, AER: 0.24
new1 prec: 0.874, rec: 0.69, F1: 0.771, AER: 0.229
new_mygd prec: 0.835, rec: 0.732, F1: 0.78, AER: 0.22
new_mygd_gdfa prec: 0.734, rec: 0.786, F1: 0.759, AER: 0.241


intersection prec: 0.818, rec: 0.268, F1: 0.404, AER: 0.596
gdfa prec: 0.508, rec: 0.448, F1: 0.476, AER: 0.524
my_gdfa prec: 0.515, rec: 0.75, F1: 0.611, AER: 0.389
argmax prec: 0.774, rec: 0.607, F1: 0.68, AER: 0.32
resnorm prec: 0.548, rec: 0.727, F1: 0.625, AER: 0.

 43%|████▎     | 3001/7000 [12:08<69:33:39, 62.62s/it]

...



intersection prec: 0.897, rec: 0.506, F1: 0.647, AER: 0.353
gdfa prec: 0.733, rec: 0.671, F1: 0.701, AER: 0.299
my_gdfa prec: 0.643, rec: 0.79, F1: 0.709, AER: 0.291
argmax prec: 0.852, rec: 0.7, F1: 0.769, AER: 0.231
resnorm prec: 0.66, rec: 0.778, F1: 0.714, AER: 0.286
itermax2-.9 prec: 0.738, rec: 0.76, F1: 0.749, AER: 0.251
itermax2-.95 prec: 0.737, rec: 0.76, F1: 0.748, AER: 0.252
itermax2-.8 prec: 0.739, rec: 0.76, F1: 0.749, AER: 0.251
my_gd prec: 0.834, rec: 0.728, F1: 0.777, AER: 0.222
my_gd_gdfa prec: 0.734, rec: 0.784, F1: 0.758, AER: 0.242
new1 prec: 0.871, rec: 0.692, F1: 0.771, AER: 0.229
new_mygd prec: 0.832, rec: 0.729, F1: 0.777, AER: 0.223
new_mygd_gdfa prec: 0.732, rec: 0.785, F1: 0.758, AER: 0.242


intersection prec: 0.818, rec: 0.268, F1: 0.404, AER: 0.596
gdfa prec: 0.508, rec: 0.448, F1: 0.476, AER: 0.524
my_gdfa prec: 0.516, rec: 0.751, F1: 0.612, AER: 0.388
argmax prec: 0.78, rec: 0.612, F1: 0.686, AER: 0.314
resnorm prec: 0.568, rec: 0.728, F1: 0.638, AER: 

 90%|█████████ | 5998/6635 [15:47<01:30,  7.04it/s]



intersection prec: 0.897, rec: 0.506, F1: 0.647, AER: 0.353
gdfa prec: 0.733, rec: 0.671, F1: 0.701, AER: 0.299
my_gdfa prec: 0.646, rec: 0.796, F1: 0.713, AER: 0.287
argmax prec: 0.858, rec: 0.699, F1: 0.77, AER: 0.229
resnorm prec: 0.609, rec: 0.791, F1: 0.688, AER: 0.312
itermax2-.9 prec: 0.745, rec: 0.764, F1: 0.754, AER: 0.246
itermax2-.95 prec: 0.743, rec: 0.763, F1: 0.753, AER: 0.247
itermax2-.8 prec: 0.746, rec: 0.764, F1: 0.755, AER: 0.245
my_gd prec: 0.831, rec: 0.738, F1: 0.782, AER: 0.218
my_gd_gdfa prec: 0.733, rec: 0.789, F1: 0.76, AER: 0.24
new1 prec: 0.874, rec: 0.696, F1: 0.775, AER: 0.225
new_mygd prec: 0.829, rec: 0.741, F1: 0.783, AER: 0.218
new_mygd_gdfa prec: 0.731, rec: 0.791, F1: 0.76, AER: 0.24


intersection prec: 0.818, rec: 0.268, F1: 0.404, AER: 0.596
gdfa prec: 0.508, rec: 0.448, F1: 0.476, AER: 0.524
my_gdfa prec: 0.512, rec: 0.749, F1: 0.608, AER: 0.392
argmax prec: 0.783, rec: 0.608, F1: 0.684, AER: 0.316
resnorm prec: 0.529, rec: 0.74, F1: 0.617, AER

100%|██████████| 6635/6635 [21:55<00:00,  5.04it/s]


train loss: 3504449.5449852943
cluster loss: 509.94970703125


intersection prec: 0.818, rec: 0.268, F1: 0.404, AER: 0.596
gdfa prec: 0.508, rec: 0.448, F1: 0.476, AER: 0.524
my_gdfa prec: 0.516, rec: 0.752, F1: 0.612, AER: 0.388
argmax prec: 0.784, rec: 0.612, F1: 0.687, AER: 0.313
resnorm prec: 0.533, rec: 0.744, F1: 0.621, AER: 0.379
itermax2-.9 prec: 0.655, rec: 0.7, F1: 0.677, AER: 0.324
itermax2-.95 prec: 0.652, rec: 0.698, F1: 0.674, AER: 0.325
itermax2-.8 prec: 0.657, rec: 0.7, F1: 0.678, AER: 0.322
my_gd prec: 0.757, rec: 0.665, F1: 0.708, AER: 0.292
my_gd_gdfa prec: 0.607, rec: 0.699, F1: 0.65, AER: 0.35
new1 prec: 0.809, rec: 0.617, F1: 0.7, AER: 0.3
new_mygd prec: 0.751, rec: 0.669, F1: 0.708, AER: 0.293
new_mygd_gdfa prec: 0.604, rec: 0.703, F1: 0.65, AER: 0.351


In [ ]:
from gnn_utils import eval_utils
importlib.reload(eval_utils)

clean_memory()
eval_utils.alignment_test(epoch, heb_test_dataset.edge_index, editf_fin, editf_heb, heb_test_verses[:50], heb_test_dataset.nodes_map,
                        dev, model, heb_test_dataset.x, pros_heb, surs_heb, heb_test_verse_alignments_inter, heb_test_verse_alignments_gdfa, writer, gnn_dataset_heb.verse_info)

eval_utils.alignment_test(epoch, grc_test_dataset.edge_index, editf_fin, editf_grc, grc_test_verses[:250], grc_test_dataset.nodes_map,
                        dev, model, grc_test_dataset.x, pros_grc, surs_grc, grc_test_verse_alignments_inter, grc_test_verse_alignments_gdfa, writer, gnn_dataset_grc.verse_info)

eval_utils.alignment_test(epoch, blinker_test_dataset.edge_index, editf12, editf22, blinker_verses[:], blinker_test_dataset.nodes_map,
                    dev, model, blinker_test_dataset.x, pros_blinker, surs_blinker, blinker_verse_alignments_inter, blinker_verse_alignments_gdfa, writer, gnn_dataset_blinker.verse_info)

clean_memory()




intersection prec: 0.858, rec: 0.272, F1: 0.413, AER: 0.587
gdfa prec: 0.534, rec: 0.474, F1: 0.502, AER: 0.498
my_gdfa prec: 0.521, rec: 0.763, F1: 0.619, AER: 0.381
argmax prec: 0.806, rec: 0.634, F1: 0.71, AER: 0.29
resnorm prec: 0.552, rec: 0.758, F1: 0.639, AER: 0.361
itermax2-.9 prec: 0.642, rec: 0.707, F1: 0.673, AER: 0.327
itermax2-.95 prec: 0.64, rec: 0.707, F1: 0.672, AER: 0.328
itermax2-.8 prec: 0.645, rec: 0.71, F1: 0.676, AER: 0.324
my_gd prec: 0.77, rec: 0.673, F1: 0.718, AER: 0.282
my_gd_gdfa prec: 0.62, rec: 0.703, F1: 0.659, AER: 0.341
new1 prec: 0.825, rec: 0.637, F1: 0.719, AER: 0.281
new_mygd prec: 0.768, rec: 0.676, F1: 0.719, AER: 0.281
new_mygd_gdfa prec: 0.619, rec: 0.706, F1: 0.66, AER: 0.34


KeyboardInterrupt: 

In [ ]:
# produce for uruba
importlib.reload(eval_utils)
editf_yor = 'yor-x-bible-2010'
editf_others = ['eng-x-bible-mixed', 'deu-x-bible-newworld', 'ces-x-bible-newworld', 'fra-x-bible-louissegond', 'hin-x-bible-newworld',
                'ita-x-bible-2009', 'prs-x-bible-goodnews', 'ron-x-bible-2006', 'spa-x-bible-newworld']

def get_pruned_verse_alignments(args):
    verse, current_editions = args
    
    #verse_aligns_inter = autils.get_verse_alignments(verse)
    verse_aligns_gdfa = autils.get_verse_alignments(verse, gdfa=True)

    #autils.prune_non_necessary_alignments(verse_aligns_inter, current_editions)
    autils.prune_non_necessary_alignments(verse_aligns_gdfa, current_editions)

    gc.collect()
    return verse_aligns_gdfa
    

verse_alignments_gdfa = {}
args = []
editfs = editf_others[:]
editfs.append(editf_yor)
for i,verse in enumerate(train_verses):
    args.append((verse, editfs))

print('going to get alignments')
with Pool(20) as p:
    all_res = p.map(get_pruned_verse_alignments, args)

for i,verse in enumerate(all_verses):
    verse_aligns_gdfa = all_res[i]
    
    verse_alignments_gdfa[verse] = verse_aligns_gdfa



for eidtf_t in editf_others:
    print('going to align blinker ')
    res = {}
    verses = set(blinker_test_dataset.nodes_map[editf_yor].keys()).intersection(blinker_test_dataset.nodes_map[eidtf_t].keys())
    res_ = eval_utils.alignment_test(epoch, blinker_test_dataset.edge_index, editf_yor, eidtf_t, list(verses), blinker_test_dataset.nodes_map,
                    dev, model, blinker_test_dataset.x, pros_blinker, surs_blinker, blinker_verse_alignments_inter, blinker_verse_alignments_gdfa, writer, gnn_dataset_blinker.verse_info)
    clean_memory()
    res.update(res_)

    print('going to align heb ')
    verses = set(heb_test_dataset.nodes_map[editf_yor].keys()).intersection(heb_test_dataset.nodes_map[eidtf_t].keys())
    res_ = eval_utils.alignment_test(epoch, heb_test_dataset.edge_index, editf_yor, eidtf_t, list(verses), heb_test_dataset.nodes_map,
                    dev, model, heb_test_dataset.x, pros_heb, surs_heb, heb_test_verse_alignments_inter, heb_test_verse_alignments_gdfa, writer, gnn_dataset_heb.verse_info)
    clean_memory()
    res.update(res_)

    print('going to align grc ')
    verses = set(grc_test_dataset.nodes_map[editf_yor].keys()).intersection(grc_test_dataset.nodes_map[eidtf_t].keys())
    res_ = eval_utils.alignment_test(epoch, grc_test_dataset.edge_index, editf_yor, eidtf_t, list(verses), grc_test_dataset.nodes_map,
                    dev, model, grc_test_dataset.x, pros_grc, surs_grc, grc_test_verse_alignments_inter, grc_test_verse_alignments_gdfa, writer, gnn_dataset_grc.verse_info)
    clean_memory()
    res.update(res_)

    print('going to align train ')
    verses = set(train_dataset.nodes_map[editf_yor].keys()).intersection(train_dataset.nodes_map[eidtf_t].keys())
    res_ = eval_utils.alignment_test(epoch, train_dataset.edge_index, editf_yor, eidtf_t, list(verses - set(masked_verses)), train_dataset.nodes_map,
                    dev, model, train_dataset.x, pros, surs, verse_alignments_inter, verse_alignments_gdfa, writer, gnn_dataset_train.verse_info)
    clean_memory()
    res.update(res_)
    
    print(f'going save alignments for {eidtf_t}')
    torch.save(res, f'/mounts/work/ayyoob/results/gnn_align/yoruba/{eidtf_t}_alignments.bin')

going to gdfa alignments
going to get alignments


In [ ]:
global model, decoder
#1/0

decoder = None
model = None

gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:

features = blinker_test_dataset.features[:]
#features_edge = train_dataset.features_edge[:]
from pprint import pprint
#print('indim',in_dim)
#features[-1].out_dim = 50
for i in features:
    #if i.type==3:
    #    i.out_dim=4
    print(vars(i))

sum(p.out_dim for p in features)
#train_dataset.features.pop()
#train_dataset.features[0] = afeatures.OneHotFeature(20, 83, 'editf')
#train_dataset.features[1] = afeatures.OneHotFeature(32, 150, 'position')
#train_dataset.features[2] = afeatures.FloatFeature(4, 'degree_centrality')
#train_dataset.features[3] = afeatures.FloatFeature(4, 'closeness_centrality')
#train_dataset.features[4] = afeatures.FloatFeature(4, 'betweenness_centrality')
#train_dataset.features[5] = afeatures.FloatFeature(4, 'load_centrality')
#train_dataset.features[6] = afeatures.FloatFeature(4, 'harmonic_centrality')
#train_dataset.features[7] = afeatures.OneHotFeature(32, 250, 'greedy_modularity_community')
##train_dataset.features.append(afeatures.MappingFeature(100, 'word'))
#torch.save(train_dataset, "/mounts/work/ayyoob/models/gnn/dataset_helfi_train_community_word.pickle")
#torch.save(train_dataset.features[-3], "./features.tmp")

In [ ]:
# count number of deleted edges by each community detection method
# from networkx.algorithms.community import greedy_modularity_communities, asyn_lpa_communities, label_propagation_communities, asyn_fluidc

# tmp_verses = [all_verses[2]]
# tmp_editions = small_editions[:10]
# tmp_dataset, tmp_nodes_map = create_dataset(tmp_verses, verse_alignments_inter, tmp_editions)

# tmp_g = pyg_utils.convert.to_networkx(tmp_dataset, to_undirected=True)
# def count_deleted_edges(tmp_dataset, c):
#     deleted_edges = 0
#     for i in range(0, len(tmp_dataset.edge_index[0]), 2):
#         for comp in c:
#             if tmp_dataset.edge_index[0][i].item() in comp and tmp_dataset.edge_index[1][i].item() not in comp:
#                 deleted_edges += 1
    
#     return deleted_edges

# print("eng token count: ", tmp_nodes_map['eng-x-bible-mixed'][tmp_verses[0]])
# print("original connected components",nx.number_connected_components(tmp_g))

# c = list(greedy_modularity_communities(tmp_g))
# print("new connected_components", len(c))
# print("deleted edges: ", count_deleted_edges(tmp_dataset, c))

# c = list(asyn_lpa_communities(tmp_g))
# print("asyn_lpa_communities number of components", len(c))
# print("deleted edges: ", count_deleted_edges(tmp_dataset, c))

# c = list(label_propagation_communities(tmp_g))
# print("label_propagation_communities number of components", len(c))
# print("deleted edges: ", count_deleted_edges(tmp_dataset, c))

# cents = nx.edge_betweenness_centrality(tmp_g)
# vals = sorted(list(cents.values()))
# print(vals[0], vals[10], vals[100], vals[1000], vals[2000], vals[3000], vals[10000])
# print(vals[-1], vals[-10], vals[-100], vals[-1000], vals[-2000], vals[-3000], vals[-10000])



In [ ]:
# measure different community detection algorithms
# from networkx.algorithms.community import greedy_modularity_communities, asyn_lpa_communities, label_propagation_communities, asyn_fluidc

# def remove_bad_community_edges(nodes_map, verses, edition_files, alignments):
#     edges_tmp = [[],[]]
#     res_edges = [[],[]]
#     for verse in verses:
#         utils.LOG.info(f"extracting edge features for {verse}")
#         for i,editf1 in enumerate(edition_files):
#             for j,editf2 in enumerate(edition_files[i+1:]):
#                 aligns = autils.get_aligns(editf1, editf2, alignments[verse])
#                 if aligns != None:
#                     for align in aligns:
#                         n1, node_count = node_nom(verse, editf1, align[0], 0, nodes_map, None, None)
#                         n2, node_count = node_nom(verse, editf2, align[1], 0, nodes_map, None, None)
#                         edges_tmp[0].extend([n1, n2])
#                         edges_tmp[1].extend([n2, n1])

#         gnx = convert_to_netx(edges_tmp)
#         print('detecting communities')
#         coms = greedy_modularity_communities(gnx)

#         print('finding good edges')
#         for i in range(0, len(edges_tmp[0]), 2):
#             for c in coms:
#                 if edges_tmp[0][i] in c and edges_tmp[0][i+1] in c:
#                     res_edges[0].extend([edges_tmp[0][i], edges_tmp[0][i+1]])
#                     res_edges[1].extend([edges_tmp[0][i+1], edges_tmp[0][i]])
#         edges_tmp = [[],[]]
#     print('to keep edges:', len(res_edges[0]))
#     return torch.tensor(res_edges, dtype=torch.long)

# # old_edge_index = train_dataset.edge_index
# # new_edge_index = remove_bad_community_edges(train_dataset.nodes_map, train_verses, small_editions, verse_alignments_inter)
# # train_dataset.edge_index = new_edge_index

# # with open("./dataset_greedy_modularity_communities.pickle", 'rb') as inf:
# #     train_dataset = pickle.load(inf)

# test_dataset = train_dataset

# print('orig edge count', old_edge_index.shape)
# print('new edge count', train_dataset.edge_index.shape)
# print("done")

In [ ]:
nodes_map = train_dataset.nodes_map
bad_edition_files = []
for edit in nodes_map:
    bad_count = 0
    for verse in nodes_map[edit]:
        if len(nodes_map[edit][verse].keys()) < 2:
            bad_count += 1
        if bad_count > 1:
            bad_edition_files.append(edit)
            break
print(bad_edition_files)

In [ ]:
all_japanese_nodes = set()
nodes_map = train_dataset.nodes_map

for bad_editionf in bad_edition_files:
    for verse in nodes_map[bad_editionf]:
        for item in nodes_map[bad_editionf][verse].items():
            all_japanese_nodes.add(item[1])

print(" all japansese nodes: ", len(all_japanese_nodes))
edge_index = train_dataset.edge_index.to('cpu')
remaining_edges_index = []
for i in tqdm(range(0, edge_index.shape[1], 2)):
    if edge_index[0, i].item() not in all_japanese_nodes and edge_index[0, i+1].item() not in all_japanese_nodes:
        remaining_edges_index.extend([i, i+1])

print('original total edges count', edge_index.shape)
print('remaining edge count', len(remaining_edges_index))
train_dataset.edge_index = edge_index[:, remaining_edges_index]
train_dataset.edge_index.shape


In [ ]:
from networkx.algorithms.community import greedy_modularity_communities, asyn_lpa_communities, label_propagation_communities, asyn_fluidc

def get_community_edges(c, verse):
    res = []
    for n1 in all_nodes_map[editf1][verse].items():
        for n2 in all_nodes_map[editf2][verse].items():
            for com in c:
                if n1[1] in com and n2[1] in com:
                    res.append((n1[0], n2[0]))
    
    return res

def intersect(e1, e2):
    res = set()
    for item in e1:
        if item in e2:
            res.add(item)
    
    return res


measures = {}
measures['intersection']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['gdfa']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c1_all']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c1_inter']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c2_all']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c2_inter']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c3_all']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c3_inter']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
numbers = [100, 300, 600, 1000, 1300, 1600, 2000, 3000, 4000, 5000]
for i in numbers:
    measures[i] = {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
    measures[i+1] = {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
coms = {}
for verse in test_verses:
    inter_edges = autils.get_aligns(editf1, editf2, verse_alignments_inter[verse])

    all_dataset, all_nodes_map = create_dataset([verse], verse_alignments_inter, small_editions)
    print("converting")
    g = pyg_utils.convert.to_networkx(all_dataset, to_undirected=True)
    print("detecting community 1")
    c1 = list(greedy_modularity_communities(g))
    print("detecting community 2")
    c2 = list(asyn_lpa_communities(g))
    print("detecting community 3")
    c3 = list(label_propagation_communities(g))

    print("detecting community 4")
    edge_betweenness = [y[0] for y in sorted(nx.edge_betweenness_centrality(g).items(), key=lambda x: x[1], reverse=True)]
    
    print('orig communities', nx.number_connected_components(g))
    prev_i = 0
    for i in numbers:
        for j in range(prev_i, i):
            g.remove_edge(*edge_betweenness[j])
        prev_i = i
        com = list(nx.connected_components(g))
        edges = get_community_edges(com, verse)
        autils.calc_and_update_alignment_score(edges, pros[verse], surs[verse], measures[i])
        autils.calc_and_update_alignment_score(intersect(edges, inter_edges), pros[verse], surs[verse], measures[i+1]) 
        print(f'communities {i}', nx.number_connected_components(g))

    c1_edges = get_community_edges(c1, verse)
    c2_edges = get_community_edges(c2, verse)
    c3_edges = get_community_edges(c3, verse)
    print('c1 communities', len(c1))
    print('c2 communities', len(c2))
    print('c3 communities', len(c3))



    autils.calc_and_update_alignment_score(inter_edges, pros[verse], surs[verse], measures['intersection'])
    autils.calc_and_update_alignment_score(autils.get_aligns(editf1, editf2, verse_alignments_gdfa[verse]), pros[verse], surs[verse], measures['gdfa'])

    autils.calc_and_update_alignment_score(c1_edges, pros[verse], surs[verse], measures['c1_all'])
    autils.calc_and_update_alignment_score(c2_edges, pros[verse], surs[verse], measures['c2_all'])
    autils.calc_and_update_alignment_score(c3_edges, pros[verse], surs[verse], measures['c3_all'])

    autils.calc_and_update_alignment_score(intersect(c1_edges, inter_edges), pros[verse], surs[verse], measures['c1_inter'])
    autils.calc_and_update_alignment_score(intersect(c2_edges, inter_edges), pros[verse], surs[verse], measures['c2_inter'])
    autils.calc_and_update_alignment_score(intersect(c3_edges, inter_edges), pros[verse], surs[verse], measures['c3_inter'])

    for item in measures:
        print(item, measures[item])
    
    
